In [ ]:
%matplotlib inline

from collections import defaultdict
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

from numpy import exp, log, sqrt
from scipy.linalg import expm, inv, solve_banded

In [ ]:
class DoubleWell():
    '''
        One-dimensional double well potential.
    '''
    def __init__(self, name='Double well', d=1, T=1, eta=1, kappa=1):
        self.name = name
        self.d = d
        self.T = T
        self.eta = eta
        self.kappa = kappa
        self.B = pt.eye(self.d).to(device)
        self.X_0 = -pt.ones(self.d).to(device)
        self.ref_sol_is_defined = False

        if self.d != 1:
            print('The double well example is only implemented for d = 1.')

    def V(self, x):
        return self.kappa * (x**2 - 1)**2

    def grad_V(self, x):
        return 4.0 * self.kappa * x * (x**2 - 1)

    def b(self, x):
        return -self.grad_V(x)

    def sigma(self, x):
        return self.B

    def f(self, x, t):
        return pt.zeros(x.shape[0]).to(device)

    def h(self, t, x, y, z):
        return -0.5 * pt.sum(z**2, dim=1)

    def g(self, x):
        return (self.eta * (x - 1)**2).squeeze()


    def compute_reference_solution(self, delta_t=0.005, xb=2.5, nx=1000):

        self.xb = xb # range of x, [-xb, xb]
        self.nx = nx # number of discrete interval
        self.dx = 2.0 * self.xb / self.nx
        self.delta_t = delta_t

        beta = 2

        self.xvec = np.linspace(-self.xb, self.xb, self.nx, endpoint=True)

        # A = D^{-1} L D
        # assumes Neumann boundary conditions

        A = np.zeros([self.nx, self.nx])
        for i in range(0, self.nx):

            x = -self.xb + (i + 0.5) * self.dx
            if i > 0:
                x0 = -self.xb + (i - 0.5) * self.dx
                x1 = -self.xb + i * self.dx
                A[i, i - 1] = -exp(beta * 0.5 * (self.V(x0) + self.V(x) - 2 * self.V(x1))) / self.dx**2
                A[i, i] = exp(beta * (self.V(x) - self.V(x1))) / self.dx**2
            if i < self.nx - 1:
                x0 = -self.xb + (i + 1.5) * self.dx
                x1 = -self.xb + (i + 1) * self.dx
                A[i, i + 1] = -exp(beta * 0.5 * (self.V(x0) + self.V(x) - 2 * self.V(x1))) / self.dx**2
                A[i, i] = A[i, i] + exp(beta * (self.V(x) - self.V(x1))) / self.dx**2

        A = -A / beta
        N = int(self.T / self.delta_t)

        D = np.diag(exp(beta * self.V(self.xvec) / 2))
        D_inv = np.diag(exp(-beta * self.V(self.xvec) / 2))

        np.linalg.cond(np.eye(self.nx) - self.delta_t * A)
        #w, vv = np.linalg.eigh(np.eye(self.nx) - self.delta_t * A)

        self.psi = np.zeros([N + 1, self.nx])
        self.psi[N, :] = exp(-self.g(self.xvec))

        for n in range(N - 1, -1, -1):
            band = - self.delta_t * np.vstack([np.append([0], np.diagonal(A, offset=1)),
                                               np.diagonal(A, offset=0) - N / self.T,
                                               np.append(np.diagonal(A, offset=1), [0])])

            self.psi[n, :] = D.dot(solve_banded([1, 1], band, D_inv.dot(self.psi[n + 1, :])))
            #psi[n, :] = np.dot(D, np.linalg.solve(np.eye(self.nx) - delta_t * A, D_inv.dot(psi[n + 1, :])));


        self.u = np.zeros([N + 1, self.nx - 1])
        for n in range(N + 1):
            for i in range(self.nx - 1):
                self.u[n, i] = -2 / beta * self.B * (- log(self.psi[n, i + 1]) + log(self.psi[n, i])) / self.dx
        #self.u = 2 / beta * np.gradient(np.log(self.psi), self.dx, 1)

    def v_true(self, x, t):
        i = np.floor((x.squeeze(0) + self.xb) / self.dx).long()
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(- log(self.psi[n, i])).reshape([1, len(i)])

    def u_true(self, x, t):
        i = np.floor((np.clip(x, -self.xb, self.xb - 2 * self.dx).squeeze(0) + self.xb) / self.dx).long()
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(self.u[n, i]).reshape([1, len(i)])
        #return interpolate.interp1d(self.xvec, self.u)(x)[:, n]
        

class DoubleWell_multidim():
    '''
        Multidimensional extension: one-dimensional double well potential in every dimension.
    '''
    def __init__(self, name='Double well', d=1, d_1=1, d_2=0, T=1, eta=1, kappa=1):
        self.name = name
        self.d = d
        self.d_1 = d_1
        self.d_2 = d_2
        self.T = T
        self.eta = eta
        self.eta_ = np.array([eta] * d_1 + [1.0] * d_2)
        self.kappa = kappa
        self.kappa_ = np.array([kappa] * d_1 + [1.0] * d_2)
        self.B = np.eye(self.d)
        self.X_0 = -np.ones(self.d)
        self.ref_sol_is_defined = False

    def V(self, x):
        return self.kappa * (x**2 - 1)**2

    def V_2(self, x):
        return (x**2 - 1)**2

    def grad_V(self, x):
        return 4.0 * np.expand_dims(self.kappa_, 1) * (x * (x**2 - np.expand_dims(np.ones(self.d), 1)))

    def b(self, x):
        return -self.grad_V(x)

    def sigma(self, x):
        return self.B # self.B.repeat(x.shape[0], 1, 1)

    def h(self, t, x, y, z):
        return -0.5 * np.sum(z**2, dim=1)

    def f(self, x, t):
        return np.zeros(x.shape[0]).to(device)

    def g_1(self, x_1):
        return self.eta * (x_1 - 1)**2

    def g_2(self, x_1):
        return (x_1 - 1)**2

    def g(self, x):
        #return (self.eta * (np.sum((x - np.ones(self.d).to(device))**2, 1))).squeeze()
        return ((np.sum(np.expand_dims(self.eta_, 1) * (x - np.expand_dims(np.ones(self.d), 1))**2, 0))).squeeze()

    def compute_reference_solution(self, delta_t=0.005, xb=2.5, nx=1000):

        self.xb = xb # range of x, [-xb, xb]
        self.nx = nx # number of discrete interval
        self.dx = 2.0 * self.xb / self.nx
        self.delta_t = delta_t

        beta = 2

        self.xvec = np.linspace(-self.xb, self.xb, self.nx, endpoint=True)

        # A = D^{-1} L D
        # assumes Neumann boundary conditions

        A = np.zeros([self.nx, self.nx])
        for i in range(0, self.nx):

            x = -self.xb + (i + 0.5) * self.dx
            if i > 0:
                x0 = -self.xb + (i - 0.5) * self.dx
                x1 = -self.xb + i * self.dx
                A[i, i - 1] = -exp(beta * 0.5 * (self.V(x0) + self.V(x) - 2 * self.V(x1))) / self.dx**2
                A[i, i] = exp(beta * (self.V(x) - self.V(x1))) / self.dx**2
            if i < self.nx - 1:
                x0 = -self.xb + (i + 1.5) * self.dx
                x1 = -self.xb + (i + 1) * self.dx
                A[i, i + 1] = -exp(beta * 0.5 * (self.V(x0) + self.V(x) - 2 * self.V(x1))) / self.dx**2
                A[i, i] = A[i, i] + exp(beta * (self.V(x) - self.V(x1))) / self.dx**2

        A = -A / beta
        N = int(self.T / self.delta_t)

        D = np.diag(exp(beta * self.V(self.xvec) / 2))
        D_inv = np.diag(exp(-beta * self.V(self.xvec) / 2))

        np.linalg.cond(np.eye(self.nx) - self.delta_t * A)
        #w, vv = np.linalg.eigh(np.eye(self.nx) - self.delta_t * A)

        self.psi = np.zeros([N + 1, self.nx])
        self.psi[N, :] = exp(-self.g_1(self.xvec))

        for n in range(N - 1, -1, -1):
            band = - self.delta_t * np.vstack([np.append([0], np.diagonal(A, offset=1)),
                                               np.diagonal(A, offset=0) - N / self.T,
                                               np.append(np.diagonal(A, offset=1), [0])])

            self.psi[n, :] = D.dot(solve_banded([1, 1], band, D_inv.dot(self.psi[n + 1, :])))
            #psi[n, :] = np.dot(D, np.linalg.solve(np.eye(self.nx) - delta_t * A, D_inv.dot(psi[n + 1, :])));


        self.u = np.zeros([N + 1, self.nx - 1])
        for n in range(N + 1):
            for i in range(self.nx - 1):
                self.u[n, i] = -2 / beta * self.B[0, 0] * (- log(self.psi[n, i + 1]) + log(self.psi[n, i])) / self.dx
        #self.u = 2 / beta * np.gradient(np.log(self.psi), self.dx, 1)

    def v_true_1(self, x, t):
        i = np.floor((x + self.xb) / self.dx).astype(int)
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(- log(self.psi[n, i])).reshape([1, len(i)])

    def u_true_1(self, x, t):
        x = np.expand_dims(x, 1)
        x = x.T
        i = np.floor((np.clip(x, -self.xb, self.xb - 2 * self.dx).squeeze(0) + self.xb) / self.dx).astype(int)
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(self.u[n, i]).reshape([1, len(i)])
        #return interpolate.interp1d(self.xvec, self.u)(x)[:, n]
        
    def compute_reference_solution_2(self, delta_t=0.005, xb=2.5, nx=1000):

        self.xb = xb # range of x, [-xb, xb]
        self.nx = nx # number of discrete interval
        self.dx = 2.0 * self.xb / self.nx
        self.delta_t = delta_t

        beta = 2

        self.xvec = np.linspace(-self.xb, self.xb, self.nx, endpoint=True)

        # A = D^{-1} L D
        # assumes Neumann boundary conditions

        A = np.zeros([self.nx, self.nx])
        for i in range(0, self.nx):

            x = -self.xb + (i + 0.5) * self.dx
            if i > 0:
                x0 = -self.xb + (i - 0.5) * self.dx
                x1 = -self.xb + i * self.dx
                A[i, i - 1] = -exp(beta * 0.5 * (self.V_2(x0) + self.V_2(x) - 2 * self.V_2(x1))) / self.dx**2
                A[i, i] = exp(beta * (self.V_2(x) - self.V_2(x1))) / self.dx**2
            if i < self.nx - 1:
                x0 = -self.xb + (i + 1.5) * self.dx
                x1 = -self.xb + (i + 1) * self.dx
                A[i, i + 1] = -exp(beta * 0.5 * (self.V_2(x0) + self.V_2(x) - 2 * self.V_2(x1))) / self.dx**2
                A[i, i] = A[i, i] + exp(beta * (self.V_2(x) - self.V_2(x1))) / self.dx**2

        A = -A / beta
        N = int(self.T / self.delta_t)

        D = np.diag(exp(beta * self.V_2(self.xvec) / 2))
        D_inv = np.diag(exp(-beta * self.V_2(self.xvec) / 2))

        np.linalg.cond(np.eye(self.nx) - self.delta_t * A)
        #w, vv = np.linalg.eigh(np.eye(self.nx) - self.delta_t * A)

        self.psi_2 = np.zeros([N + 1, self.nx])
        self.psi_2[N, :] = exp(-self.g_2(self.xvec))

        for n in range(N - 1, -1, -1):
            band = - self.delta_t * np.vstack([np.append([0], np.diagonal(A, offset=1)),
                                               np.diagonal(A, offset=0) - N / self.T,
                                               np.append(np.diagonal(A, offset=1), [0])])

            self.psi_2[n, :] = D.dot(solve_banded([1, 1], band, D_inv.dot(self.psi_2[n + 1, :])))
            #psi[n, :] = np.dot(D, np.linalg.solve(np.eye(self.nx) - delta_t * A, D_inv.dot(psi[n + 1, :])));


        self.u_2 = np.zeros([N + 1, self.nx - 1])
        for n in range(N + 1):
            for i in range(self.nx - 1):
                self.u_2[n, i] = -2 / beta * self.B[0, 0] * (- log(self.psi_2[n, i + 1]) + log(self.psi_2[n, i])) / self.dx
        #self.u = 2 / beta * np.gradient(np.log(self.psi), self.dx, 1)
        
    def v_true_2(self, x, t):
        i = np.floor((x + self.xb) / self.dx).astype(int)
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(- log(self.psi_2[n, i])).reshape([1, len(i)])

    def u_true_2(self, x, t):
        x = np.expand_dims(x, 1)
        x = x.T
        i = np.floor((np.clip(x, -self.xb, self.xb - 2 * self.dx).squeeze(0) + self.xb) / self.dx).astype(int)
        i[-1] -= 2
        n = int(np.ceil(t / self.delta_t))
        return np.array(self.u_2[n, i]).reshape([1, len(i)])
        #return interpolate.interp1d(self.xvec, self.u)(x)[:, n]

    def v_true(self, x, t):
        return np.concatenate([self.v_true_1(x[i, :], t).T for i in range(self.d_1)] + [self.v_true_2(x[i, :], t).T for i in range(self.d_1, self.d)], 1).T

    def u_true(self, x, t):
        return np.concatenate([self.u_true_1(x[i, :], t).T for i in range(self.d_1)] + [self.u_true_2(x[i, :], t).T for i in range(self.d_1, self.d)], 1).T


In [ ]:
def importance_sampling(dw, K, delta_t, epsilon=0, lambda_=1, alpha=100, verbose=True):

    sq_delta_t = np.sqrt(delta_t)
    N = int(dw.T / delta_t)

    X = np.repeat(np.expand_dims(dw.X_0, 1), K, 1)
    X_u = np.repeat(np.expand_dims(dw.X_0, 1), K, 1)
    ito_int = np.zeros(K)
    riemann_int = np.zeros(K)
    f_int = np.zeros(K)
    f_int_u = np.zeros(K)

    for n in range(N + 1):
        xi = np.random.randn(dw.d, K)
        X = X + dw.b(X) * delta_t + dw.sigma(X).dot(xi) * sq_delta_t
        if n * delta_t >= 0:
            #ut = dw.u_true(X_u, n * delta_t) + epsilon * np.sin(alpha * X_u)
            #ut = dw.u_true(X_u, n * delta_t) + epsilon * np.sin(alpha * n * delta_t)
            ut = dw.u_true(X_u, n * delta_t) * lambda_
        else:
            ut = dw.u_true(X_u, n * delta_t)

        X_u = X_u + (dw.b(X_u) + dw.sigma(X).dot(ut)) * delta_t + dw.sigma(X).dot(xi) * sq_delta_t
        ito_int += np.sum(ut * xi, 0) * sq_delta_t
        riemann_int += np.sum(ut**2, 0) * delta_t

    girsanov = np.exp(- ito_int - 0.5 * riemann_int)

    stats = {}
    stats['naive'] = {}
    stats['IS'] = {}
    stats['naive']['mean'] = np.mean(np.exp(- f_int - dw.g(X)))
    stats['naive']['variance'] = np.var(np.exp(- f_int - dw.g(X)))
    stats['naive']['relative_error'] = np.sqrt(stats['naive']['variance']) / stats['naive']['mean']
    stats['IS']['mean'] = np.mean(np.exp(- f_int_u - dw.g(X_u)) * girsanov)
    stats['IS']['variance'] = np.var(np.exp(- f_int_u - dw.g(X_u)) * girsanov)
    stats['IS']['relative_error'] = np.sqrt(stats['IS']['variance']) / stats['IS']['mean']
    
    if verbose:
        print('naive mean: %.4e, naive variance: %.4e, naive RE: %.4e' % (stats['naive']['mean'], stats['naive']['variance'], stats['naive']['relative_error']))
        print('IS mean:    %.4e, IS variance:    %.4e, IS RE:    %.4e' % (stats['IS']['mean'], stats['IS']['variance'], stats['IS']['relative_error']))
        print('true mean:  %.4e' % np.exp(-np.sum(dw.v_true(np.expand_dims(dw.X_0, 1), 0))))
    return stats

In [ ]:
dw_1 = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=1, eta=1, kappa=1)
dw_1.compute_reference_solution(delta_t=0.001, nx=3000)
dw_2 = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=1, eta=2, kappa=2)
dw_2.compute_reference_solution(delta_t=0.001, nx=3000)
dw_3 = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=1, eta=3, kappa=3)
dw_3.compute_reference_solution(delta_t=0.001, nx=3000)

In [ ]:
np.random.seed(123)

lambda_range = np.linspace(1, 1.5, 20)

RE_1 = []
RE_2 = []
RE_3 = []

for lambda_ in lambda_range:
    stats = importance_sampling(dw_1, K=1000000, delta_t=0.0001, lambda_=lambda_, verbose=False)
    RE_1.append(stats['IS']['relative_error'])
    stats = importance_sampling(dw_2, K=1000000, delta_t=0.0001, lambda_=lambda_, verbose=False)
    RE_2.append(stats['IS']['relative_error'])
    stats = importance_sampling(dw_3, K=1000000, delta_t=0.0001, lambda_=lambda_, verbose=False)
    RE_3.append(stats['IS']['relative_error'])

In [ ]:
E = 14 # seed 123
plt.plot(lambda_range[:E], RE_1[:E], label=r'$\rho = 1, \kappa = 1$');
plt.plot(lambda_range[:E], RE_2[:E], label=r'$\rho = 2, \kappa = 2$');
plt.plot(lambda_range[:E], RE_3[:E], label=r'$\rho = 3, \kappa = 3$');
plt.legend()
plt.xlabel(r'$\zeta$')
plt.ylabel(r'$r(u)$');
#plt.plot(epsilon_range, np.sqrt(np.exp((1 + np.sqrt(2)) * d * epsilon_range**2 * 0.2) - 1));
#plt.plot(epsilon_range[:E], np.sqrt(np.exp(dw_.d * epsilon_range[:E]**2 * dw_.T) - 1));

In [ ]:
dw_1 = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=1, eta=1, kappa=1)
dw_1.compute_reference_solution(delta_t=0.001)
dw_1.compute_reference_solution_2(delta_t=0.001)

dw_2 = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=1, eta=3, kappa=5)
dw_2.compute_reference_solution(delta_t=0.001)
dw_2.compute_reference_solution_2(delta_t=0.001)

In [ ]:
IS = importance_sampling(dw=dw_1, K=10000, delta_t=0.001, epsilon=1.0, verbose=True)

In [ ]:
IS = importance_sampling(dw=dw_2, K=10000, delta_t=0.001, epsilon=1.0, verbose=True)

In [ ]:
RE_doublewell = np.load('RE_doublewell.npy')

In [ ]:
epsilon = 0.5

COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown',
          'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

def g(x, nu):
    return nu * (x - 1)**2

def psi(x, kappa):
    return kappa * (x**2 - 1)**2

x_l = -2
x_r = 2

x_val = np.linspace(x_l - 0.5, x_r + 0.5, 3000)

fig, ax = plt.subplots(2, 2, figsize=(8, 7))

ax[0, 0].plot(x_val, psi(x_val, 1), label=r'potential $\psi$');
#ax[0, 0].plot(x_val, np.exp(-g(x_val, 1)), label=r'target $e^{-g}$')
ax[0, 0].fill_between(x_val, np.exp(-g(x_val, 1)), step='pre', alpha=0.4)
ax[0, 0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[0, :]), color=COLORS[1], label=r'optimal, $t = 0$');
ax[0, 0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[0, :]) + x_val * epsilon, '--', color=COLORS[1], label=r'perturbed, $t = 0$');
ax[0, 0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[-1, :]), color=COLORS[2], label=r'optimal, $t = T$');
ax[0, 0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[-1, :]) + x_val * epsilon, '--', color=COLORS[2], label=r'perturbed, $t = T$');
ax[0, 0].set_xlabel(r'$x$');
ax[0, 0].set_xticks(np.linspace(-2, 2, 5))
ax[0, 0].legend()

ax[0, 1].plot(x_val, psi(x_val, 5));
#ax[0, 1].plot(x_val, np.exp(-g(x_val, 3)))
ax[0, 1].fill_between(x_val, np.exp(-g(x_val, 3)), step='pre', alpha=0.4, label=r'target $e^{-g}$')
ax[0, 1].plot(x_val_2, psi(x_val, 5) - np.log(dw_2.psi[0, :]), color=COLORS[1], label=r'optimal potential at $t = 0$');
ax[0, 1].plot(x_val_2, psi(x_val, 5) - np.log(dw_2.psi[0, :]) + x_val_2 * epsilon, '--', color=COLORS[1], label=r'perturbed potential at $t = 0$');
ax[0, 1].plot(x_val_2, psi(x_val, 5) - np.log(dw_2.psi[-1, :]), color=COLORS[2], label=r'optimal potential at $t = T$');
ax[0, 1].plot(x_val_2, psi(x_val, 5) - np.log(dw_2.psi[-1, :]) + x_val_2 * epsilon, '--', color=COLORS[2], label=r'perturbed potential at $t = T$');
ax[0, 1].set_xlabel(r'$x$');
ax[0, 1].set_xticks(np.linspace(-2, 2, 5))


ax[0, 0].set_xlim(x_l, x_r)
ax[0, 1].set_xlim(x_l, x_r)
ax[0, 0].set_ylim(0, 12)
ax[0, 1].set_ylim(0, 12)
ax[0, 0].set_title(r'Potentials, $\rho = 1, \kappa = 1, \varepsilon = %.1f$' % epsilon)
ax[0, 1].set_title(r'Potentials, $\rho = 3, \kappa = 3, \varepsilon = %.1f$' % epsilon)
#ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5)); #  loc='center left', bbox_to_anchor=(1, 0.5)


nus = np.linspace(0.1, 5, 100)
kappas = np.linspace(0.1, 5, 100)
x = nus
y = kappas
xv, yv = np.meshgrid(x, y)
xy = np.vstack((xv.flatten(), yv.flatten())).T

Y = RE_doublewell

p = ax[1, 0].imshow(Y, cmap=cm.jet, extent=[nus[0], nus[-1], kappas[0], kappas[-1]], vmin=np.min(Y), vmax=np.max(Y),
                        origin='lower', interpolation='none', norm=LogNorm(vmin=0.01, vmax=1));
plt.colorbar(p, ax=ax[1, 0]);
ax[1, 0].set_xlabel(r'$\rho$')
ax[1, 0].set_ylabel(r'$\kappa$');
ax[1, 0].set_title('relative error of naive estimator')

ax[1, 1].plot(lambda_range[:E], RE_1[:E], label=r'$\rho = 1, \kappa = 1$');
ax[1, 1].plot(lambda_range[:E], RE_2[:E], label=r'$\rho = 2, \kappa = 2$');
ax[1, 1].plot(lambda_range[:E], RE_3[:E], label=r'$\rho = 3, \kappa = 3$');
ax[1, 1].legend()
ax[1, 1].set_xlabel(r'$\zeta$')
ax[1, 1].set_ylabel(r'$r(u)$');
ax[1, 1].set_title('relative errors for multiplicative perturbation')

fig.tight_layout()
#fig.savefig('img/double_well_1d_pertubation_RE_2.pdf')

In [ ]:
epsilon = 0.5

COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown',
          'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

def g(x, nu):
    return nu * (x - 1)**2

def psi(x, kappa):
    return kappa * (x**2 - 1)**2

x_l = -2
x_r = 2

x_val = np.linspace(x_l - 0.5, x_r + 0.5, 1000)
x_val_2 = np.linspace(x_l - 0.5, x_r + 0.5, 3000)

fig, ax = plt.subplots(1, 4, figsize=(15, 3.2))

ax[0].plot(x_val, psi(x_val, 1), label=r'potential $\psi$');
#ax[0].plot(x_val, np.exp(-g(x_val, 1)), label=r'target $e^{-g}$')
ax[0].fill_between(x_val, np.exp(-g(x_val, 1)), step='pre', alpha=0.4)
ax[0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[0, :]), color=COLORS[1], label=r'optimal, $t = 0$');
ax[0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[0, :]) + x_val * epsilon, '--', color=COLORS[1], label=r'perturbed, $t = 0$');
ax[0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[-1, :]), color=COLORS[2], label=r'optimal, $t = T$');
ax[0].plot(x_val, psi(x_val, 1) - np.log(dw_1.psi[-1, :]) + x_val * epsilon, '--', color=COLORS[2], label=r'perturbed, $t = T$');
ax[0].set_xlabel(r'$x$');
ax[0].set_xticks(np.linspace(-2, 2, 5))
ax[0].legend()

ax[1].plot(x_val, psi(x_val, 5));
#ax[1].plot(x_val, np.exp(-g(x_val, 3)))
ax[1].fill_between(x_val, np.exp(-g(x_val, 3)), step='pre', alpha=0.4, label=r'target $e^{-g}$')
ax[1].plot(x_val_2, psi(x_val_2, 5) - np.log(dw_2.psi[0, :]), color=COLORS[1], label=r'optimal potential at $t = 0$');
ax[1].plot(x_val_2, psi(x_val_2, 5) - np.log(dw_2.psi[0, :]) + x_val_2 * epsilon, '--', color=COLORS[1], label=r'perturbed potential at $t = 0$');
ax[1].plot(x_val_2, psi(x_val_2, 5) - np.log(dw_2.psi[-1, :]), color=COLORS[2], label=r'optimal potential at $t = T$');
ax[1].plot(x_val_2, psi(x_val_2, 5) - np.log(dw_2.psi[-1, :]) + x_val_2 * epsilon, '--', color=COLORS[2], label=r'perturbed potential at $t = T$');
ax[1].set_xlabel(r'$x$');
ax[1].set_xticks(np.linspace(-2, 2, 5))


ax[0].set_xlim(x_l, x_r)
ax[1].set_xlim(x_l, x_r)
ax[0].set_ylim(0, 12)
ax[1].set_ylim(0, 12)
ax[0].set_title(r'Potentials, $\rho = 1, \kappa = 1, \varepsilon = %.1f$' % epsilon)
ax[1].set_title(r'Potentials, $\rho = 3, \kappa = 3, \varepsilon = %.1f$' % epsilon)
#ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5)); #  loc='center left', bbox_to_anchor=(1, 0.5)


nus = np.linspace(0.1, 5, 100)
kappas = np.linspace(0.1, 5, 100)
x = nus
y = kappas
xv, yv = np.meshgrid(x, y)
xy = np.vstack((xv.flatten(), yv.flatten())).T

Y = RE_doublewell

p = ax[2].imshow(Y, cmap=cm.jet, extent=[nus[0], nus[-1], kappas[0], kappas[-1]], vmin=np.min(Y), vmax=np.max(Y),
                        origin='lower', interpolation='none', norm=LogNorm(vmin=0.01, vmax=1));
plt.colorbar(p, ax=ax[2]);
ax[2].set_xlabel(r'$\rho$')
ax[2].set_ylabel(r'$\kappa$');
ax[2].set_title('relative error of naive estimator')

ax[3].plot(lambda_range[:E], RE_1[:E], label=r'$\rho = 1, \kappa = 1$');
ax[3].plot(lambda_range[:E], RE_2[:E], label=r'$\rho = 2, \kappa = 2$');
ax[3].plot(lambda_range[:E], RE_3[:E], label=r'$\rho = 3, \kappa = 3$');
ax[3].legend()
ax[3].set_xlabel(r'$\zeta$')
ax[3].set_ylabel(r'$r(u)$');
ax[3].set_title('relative errors for multiplicative perturbation')


fig.tight_layout()
#fig.savefig('img/double_well_1d_pertubation_RE.pdf')

In [ ]:
x_val = np.linspace(x_l - 0.5, x_r + 0.5, 1000)

plt.plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0).T)
plt.plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0).squeeze() + 0.2 * np.sin(x_val * 50));
plt.plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0.99).T)
plt.plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0.99).squeeze() + 0.2 * np.sin(x_val * 50));

In [ ]:
epsilon_range = np.linspace(0, 2.5, 10)

RE_sin_t = []
#RE_sin_x = []

for epsilon in epsilon_range:
    stats = importance_sampling(dw=dw_1, K=100000, delta_t=0.001, epsilon=epsilon, verbose=False, alpha=50)
    RE_sin_t.append(stats['IS']['relative_error'])
    #RE_sin_x.append(stats['IS']['relative_error'])

In [ ]:
E = 9
alpha = 50

fig, ax = plt.subplots(1, 1, figsize=(5, 3.5))
ax.plot(epsilon_range[:E], sqrt(exp(epsilon_range[:E]**2 * (dw_1.T / 2 - np.sin(2 * alpha * dw_1.T) / (4 * alpha))) - 1), '--', label='time perturbation formula');
ax.plot(epsilon_range[:E], RE_sin_t[:E], label='time perturbation sampled');
ax.plot(epsilon_range[:E], RE_sin_x[:E], label='space perturbation');

ax.legend();


In [ ]:
epsilon = 0.2
alpha = 50.0
x = -0.5
T = 1

t_val = np.linspace(0, 1, 1000)

fig, ax = plt.subplots(1, 3, figsize=(14, 4))

ax[0].set_title(r'optimal control, pertubation in $t$')
ax[0].plot(t_val, [dw_1.u_true(np.array([[x]]), t)[0, 0] for t in t_val], color=COLORS[0], label=r'optimal, $x=-0.5$');
ax[0].plot(t_val, [dw_1.u_true(np.array([[x]]), t)[0, 0] + epsilon * np.sin(alpha * t) for t in t_val], '--', color=COLORS[0], 
           label=r'perturbed, $x=-0.5$');

ax[0].plot(t_val, [dw_1.u_true(np.array([[0.0]]), t)[0, 0] for t in t_val], color=COLORS[1], label=r'optimal, $x=0$');
ax[0].plot(t_val, [dw_1.u_true(np.array([[0.0]]), t)[0, 0] + epsilon * np.sin(alpha * t) for t in t_val], '--', color=COLORS[1],
          label=r'perturbed, $x=0$');

ax[0].set_xlabel(r'$t$')
ax[0].legend(); 

ax[1].set_title(r'optimal control, pertubation in $x$')

ax[1].plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0).T, color=COLORS[0], label=r'optimal, $t=0$')
ax[1].plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0).squeeze() + epsilon * np.sin(x_val * alpha), '--', color=COLORS[0],
           label=r'perturbed, $t=0$');

ax[1].plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0.9).T, color=COLORS[1], label=r'optimal, $t=0.9$')
ax[1].plot(x_val, dw_1.u_true(np.expand_dims(x_val, 0), 0.9).squeeze() + epsilon * np.sin(x_val * alpha), '--', color=COLORS[1],
          label=r'perturbed, $t=0.9$');

ax[1].set_xlabel(r'$x$')

ax[1].legend(); #  loc='center left', bbox_to_anchor=(1, 0.5)

alpha = 50

ax[2].plot(epsilon_range[:E], sqrt(exp(epsilon_range[:E]**2 * (dw_1.T / 2 - np.sin(2 * alpha * dw_1.T) / (4 * alpha))) - 1),
          '--', label=r'time perturbation formula');
ax[2].plot(epsilon_range[:E], RE_sin_t[:E], label=r'time perturbation sampled');
ax[2].plot(epsilon_range[:E], RE_sin_x[:E], label=r'space perturbation sampled');
ax[2].legend();
ax[2].set_title('relative error')
ax[2].set_xlabel(r'$\varepsilon$');

fig.tight_layout(rect=[0, 0.03, 1, 0.93])

#fig.savefig('img/double_well_sine_pertubations.pdf')

## plot for illustration of rare events

In [ ]:
dw = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=10, eta=3, kappa=5)
dw.compute_reference_solution(delta_t=0.001)

In [ ]:
dw = DoubleWell_multidim(d=1, d_1=1, d_2=0, T=10, eta=3, kappa=5)
dw.compute_reference_solution(delta_t=0.001)

delta_t = 0.01
sq_delta_t = np.sqrt(delta_t)
N = int(dw.T / delta_t)
K = 1000000

X = np.repeat(np.expand_dims(dw.X_0, 1), K, 1)
X_u = np.repeat(np.expand_dims(dw.X_0, 1), K, 1)
ito_int = np.zeros(K)
riemann_int = np.zeros(K)
f_int = np.zeros(K)
f_int_u = np.zeros(K)

for n in range(N + 1):
    xi = np.random.randn(dw.d, K)
    X = X + dw.b(X) * delta_t + dw.sigma(X).dot(xi) * sq_delta_t
    ut = dw.u_true(X_u, n * delta_t)

    X_u = X_u + (dw.b(X_u) + dw.sigma(X).dot(ut)) * delta_t + dw.sigma(X).dot(xi) * sq_delta_t
    ito_int += np.sum(ut * xi, 0) * sq_delta_t
    riemann_int += np.sum(ut**2, 0) * delta_t

girsanov = np.exp(- ito_int - 0.5 * riemann_int)


In [ ]:
dw.psi.shape

In [ ]:
COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown',
          'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

def g(x, nu):
    return nu * (x - 1)**2

def psi(x, kappa):
    return kappa * (x**2 - 1)**2

x_l = -2#-1.52
x_r = 2#1.52

x_val = np.linspace(x_l, x_r, 1000)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
#ax2 = ax.twinx()
ax.plot(x_val, psi(x_val, 5), label=r'Potential $\Psi$');
ax.plot(x_val, psi(x_val, 5) + g(x_val, 3), color=COLORS[2], label=r'optimal potential at $t = T$');
ax.fill_between(x_val, np.exp(-g(x_val, 3)), step='pre', alpha=0.4, label=r'target $e^{-g}$', color=COLORS[1])
ax.set_ylim(0, 10)
ax.set_xlim(x_l, x_r);
ax.hist(X.squeeze(), histtype='stepfilled', bins=500, normed=True, label=r'distribution of $X_T$', color=COLORS[0], alpha=0.7);
ax.hist(X_u.squeeze(), histtype='stepfilled', bins=500, normed=True, label=r'distribution of $X^{u^*}_T$', color=COLORS[2], alpha=0.7);
ax.set_xlabel(r'$x$')
ax.legend();

#fig.savefig('img/double_well_rare_events.pdf')